In [3]:
!python --version

Python 3.9.17


In [4]:
!pip install boto3 --upgrade

In [8]:
from langchain.retrievers import AmazonKendraRetriever
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
import os
import boto3

In [6]:
# %env KENDRA_INDEX_ID="f8172c74-6d74-499e-9ff7-7882618cb269"
# %env MODEL_ENDPOINT="jumpstart-dft-hf-eqa-bert-large-cased"
# print(os.environ["AWS_REGION"])
# print(os.environ["KENDRA_INDEX_ID"])
# print(os.environ["MODEL_ENDPOINT"])

In [10]:
region = os.environ["AWS_REGION"]
kendra_client = boto3.client('kendra', region_name=region)

retriever = AmazonKendraRetriever(index_id="f8172c74-6d74-499e-9ff7-7882618cb269", client=kendra_client)

In [45]:
question = "What is this document about"
docs = retriever.get_relevant_documents(question)
# print(docs)
# print(docs[0].page_content)

page_contents = [doc.page_content for doc in docs]
combined_content = ', '.join(page_contents)
# print(combined_content)

question_context = [question, combined_content]
encoded_text = json.dumps(question_context).encode('utf-8')

endpoint_name = 'jumpstart-dft-hf-eqa-bert-large-cased'
sagemaker_client = boto3.client('runtime.sagemaker')
response = sagemaker_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/list-text', Body=encoded_text)

model_predictions = json.loads(response['Body'].read())
answer = model_predictions['answer']
print(answer)

Transfer of land
